## Computational Chemistry for Experimentalists
## Module 8: Self-Consistent Field Calculations  

Molecular orbital theory builds a separate orbital for each electron in a molecule. Each orbital must depend on the average electron distribution, and therefore depend on all of the other orbitals. This nonlinear equation is solved self-consistently, iterating back and forth between a set of guess orbitals, a guess wavefunction and electron distrbution, and a new guess of orbitals. These examples show how that SCF procedure can work and not work. 

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import math
import numpy
import matplotlib.pyplot as plt
from pyscf import gto,scf,dft,cc
from pyscf.tools import cubegen
import py3Dmol

## Example 1:Densities from Each SCF Cycle  

Here we compue the electr densities from each step of an SCF cycle, We see that the densities converge rapidly with cycle.  

In [ ]:
m3=Chem.MolFromSmiles('O=C=O')
m2=Chem.AddHs(m3)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)
mb1=Chem.MolToMolBlock(m2)
elements = [atom.GetSymbol() for atom in m2.GetAtoms()]
coordinates = m2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]
m = gto.Mole(basis="cc-pvtz")
m.atom = atoms
m.build();
mf=scf.RHF(m)
mf.max_cycle=1
mf.kernel() 
cubegen.density(m, f'co_rho1.cube', mf.make_rdm1())
mf=scf.RHF(m)
mf.max_cycle=2
mf.kernel() 
cubegen.density(m, f'co_rho2.cube', mf.make_rdm1())
mf.max_cycle=5
mf.kernel() 
cubegen.density(m, f'co_rho3.cube', mf.make_rdm1())
with open(f"co_rho1.cube") as f:
    r1 = f.read()
with open(f"co_rho2.cube") as f:
    r2 = f.read()
with open(f"co_rho3.cube") as f:
    r3 = f.read()
p = py3Dmol.view(width=600,height=200,viewergrid=(1,3))
p.addModel(mb1, 'sdf',viewer=(0,0))
p.addModel(mb1, 'sdf',viewer=(0,1))
p.addModel(mb1, 'sdf',viewer=(0,2))
p.addVolumetricData(r1, "cube", {'isoval': 0.02, 'color': "gray", 'opacity': 0.75},viewer=(0,0))
p.addVolumetricData(r2, "cube", {'isoval': 0.02, 'color': "gray", 'opacity': 0.75},viewer=(0,1))
p.addVolumetricData(r3, "cube", {'isoval': 0.02, 'color': "gray", 'opacity': 0.75},viewer=(0,2))
p.setStyle({'stick':{},'sphere':{"scale":0.1}})
p.show()
p.png()

## Example 2: Converging and Non-Converging SCF  

Self-consistent field calculations converge easily for "normal" systems, but can break down for near-degeneracies. This example shows that for carbon monoxide

In [ ]:
%%capture cap 
m1=gto.Mole(atom='C 0.0 0.0 0.0; O 0.0 0.0 1.2',basis='3-21g')
m1.build()
mf1=scf.RHF(m1)
mf1.verbose=4
mf1.kernel()

In [ ]:
with open('COSCF1.txt', 'w') as file:
    file.write(cap.stdout)
E1s=[]
with open('COSCF1.txt', 'r') as file:
    for line in file:
        fields=line.split()
        if(len(fields)>0):
            if(fields[0]=='cycle='):
                E=(float(fields[3]))
                E1s.append(E)

In [ ]:
E1s

In [ ]:
%%capture cap 
m=gto.Mole(atom='C 0.0 0.0 0.0; O 0.0 0.0 1.8',basis='3-21g')
m.build()
mf2=scf.RHF(m)
mf2.verbose=4
mf2.kernel()

In [ ]:
with open('COSCF2.txt', 'w') as file:
    file.write(cap.stdout)
E2s=[]
with open('COSCF2.txt', 'r') as file:
    for line in file:
        fields=line.split()
        if(len(fields)>0):
            if(fields[0]=='cycle='):
                E=(float(fields[3]))
                E2s.append(E)

In [ ]:
%%capture cap 
m=gto.Mole(atom='C 0.0 0.0 0.0; O 0.0 0.0 3.8',basis='3-21g')
m.build()
mf2=scf.RHF(m)
mf2.verbose=4
mf2.kernel()

In [ ]:
with open('COSCF3.txt', 'w') as file:
    file.write(cap.stdout)
E3s=[]
with open('COSCF3.txt', 'r') as file:
    for line in file:
        fields=line.split()
        if(len(fields)>0):
            if(fields[0]=='cycle='):
                E=(float(fields[3]))
                E3s.append(E)

In [ ]:
E3s

In [ ]:
E1sb=numpy.log10(0.0000000000000000001+numpy.abs(E1s-E1s[len(E1s)-1]*numpy.ones(len(E1s))))
E2sb=numpy.log10(0.0000000000000000001+numpy.abs(E2s-E2s[len(E2s)-1]*numpy.ones(len(E2s))))
E3sb=numpy.log10(0.0000000000000000001+numpy.abs(E3s-E3s[len(E3s)-1]*numpy.ones(len(E3s))))

In [ ]:
fig=plt.figure()
ax1=fig.add_subplot(111)
plt.ylim(-10,2)
ax1.plot(range(len(E1s)),E1sb,'-o',label="1.2 Angstrom")
ax1.plot(range(len(E2s)),E2sb,'-x',label="1.8 Angstrom")
ax1.plot(range(len(E3s)),E3sb,'-*',label="3.8 Angstrom")

## Example 3: Spin Symmetry Breaking 

Symmetry breaking removes artefacts due to strong correlation, but is not a panacea. 

In [ ]:
basis='3-21g'
rs=[10,9,8,7,6,5,4,3,2.8,2.6,2.4,2.2,2.0,1.8,1.6,1.4,1.2,1.0,0.8,0.7,0.6,0.5]
m.build()
mf=scf.UHF(m)
mf.kernel()
EH=mf.e_tot
EUHF=numpy.zeros_like(rs)
ERHF=numpy.zeros_like(rs)
i=-1
Pguess=None
for r in rs:
    i=i+1
    geom='H 0.0 0.0 0.0; H 0.0 0.0 %.3f '%(r)
    m=gto.Mole(atom=geom)
    m.build()
    mf=scf.RHF(m)
    mf.kernel()
    ERHF[i]=627.5095*(mf.e_tot-2*EH)
    P=mf.make_rdm1()
    if(Pguess is None):
        Pguess=numpy.array([P,P])
        Pguess[0,0,0]=Pguess[0,0,0]+0.1
        Pguess[1,0,0]=Pguess[1,0,0]-0.1
    mf2=scf.UHF(m)
    mf2.kernel(dm0=Pguess)
    Pguess=mf2.make_rdm1()
    EUHF[i]=627.5095*(mf2.e_tot-2*EH)

In [ ]:
fig=plt.figure()
ax1=fig.add_subplot(111)
#plt.ylim(-10,2)
ax1.plot(rs,ERHF,'-o',label="RHD")
ax1.plot(rs,EUHF,'-x',label="UHF")

Assignment: 

(1) Compute the HOMO-LUMO gap of H2 using RHF and UHF, as a function of bond length.

(2) Compute the rotational potential surface for ethylene using restricted and unrestricted Hartree-Fock theory. Compare to literature calculations, and explain the problems with RHF and with UHF.